Apollo路径规划模块文档：https://github.com/ApolloAuto/apollo/tree/master/modules/planning

规划的第一步，是路线导航，侧重于如何从地图上的A前往B，在进行路线规划时，将地图数据作为输入，并输出可行驶路径；

手机导航系统是路径规划的一个示例；

在Apollo中通过路线规划模块处理该任务，一旦构建了高水平的路线，我们就可以放大至轨迹规划；

通过轨迹规划，可以做出微妙的决策，以避开障碍物，并为乘客创造平稳的乘车体验；Apollo中通过规划处理模块处理该任务；

轨迹规划的目标是生产免碰撞和舒适的可执行轨迹，该轨迹由一系列点定义，每个点都有一个关联速度，
和一个指示合适应抵达那个点的时间戳；

路线规划目标是，找到从地图上的A前往B的最佳路径；

输入为：地图，起点，目的点
apollo提供地图数据包括公路网和实时交通信息，

输出，最佳行驶路径；

## 世界到图

Apollo在搜索最佳路径之前，将地图数据重新格式化为“图形”的数据结构，该图形由“节点”和“边缘”组成；

节点代表路段，边缘代表这些路段之间的连接，

![](./plan01.png)

我们可以对从一个节点移动到另一个节点所需的成本进行建模；
比如拐过交叉路口一般比执行更费时，所以节点1到节点4的成本，高于节点1到节点3的成本；


将地图转换为图形的好处在于，在计算机科学领域中，人们已发现许多用于在图形中查找路径的快速算法；

一旦在图形中找到最佳路径，就可以轻松地将图形中的路径重新转换为地图上的路径；

$A\start$ 是经典的路径查找处理算法；

从路由到轨迹

高等级地图路线只是规划过程的一部分，我们仍需要构建沿这条路线前进的低等级轨迹，这意味着要处理不属于地图的物体，
如其他车辆、行人等，即与公路上其他物体“互动”。

![](./plan02.png)

这些场景需要更低级别，更高精确度的规划，这一级别的规划，叫做“轨迹生成”。


## 3D轨迹

轨迹生成的目标是，生成由一系列路径点所定义的轨迹。

为每个路径点分配了一个时间戳和速度，让一条曲线与这些路径点拟合，生成轨迹的几何表征；

由于移动的障碍物可能会暂时阻挡部分路段，轨迹中的每个路点都有时间戳，我们可以将时间戳与预测模块的输出相结合；

![](./plan03.png)

以确保我们在通过时，轨迹上的每个路径点均未被占用，这些时间戳创建了一个三维轨迹，每个路径点由空间的两个维度以及时间维度组成；

还为每个路径点指定了一个速度，速度用于确保车辆按时到达每个路径点。


## 轨迹评估

车辆最终是否和道路中心线平行对齐；

轨迹上每一点距离障碍物的距离；

在实施轨迹时我们的其策划是否会超过速度限制；

乘客的舒适感；

### 约束

首先，轨迹应避免碰撞；

其次，乘客感到舒适；

所以路径点之间的过渡以及速度的变化必须平滑；

再者，路径点对车辆应实际可运行；

最后，轨迹应合法；确保轨迹遵守每个路径点法律法规



## 损失函数

任何两点之间，可能会有多个不会碰撞，舒适，可行且合法的轨迹，如何选择最佳轨迹呢？

成本函数：

为每个轨迹分配一个“成本”，选择成本最低的轨迹，轨迹“成本”由各种犯规处罚组成，
如，偏离道路中心，有可能会碰撞，超速，或轨迹的曲率和加速度让乘客不适，都会使成本增加；
轨迹成本将所有这些缺陷聚合为单个数字，这样可以对不同轨迹排名

![](./plan04.png)

车辆甚至可能在不同的环境中使用不同的成本函数，例如告诉公路的成本函数与停车场的成本函数不同；

## Frenet坐标系

Frenet坐标系描述了汽车相对于道路的位置，纵坐标为S，代表沿道路的距离,横坐标为d,表示纵向线的位移(偏离中心线的距离)；
在每个道路点上，s和d都是垂直的。

![](./plan05.png)

## 路径-速度解耦规划

分两步：路径规划和速度规划

首先，在路径规划生成的候选曲线中，使用成本函数评估，选择最佳路径曲线；

下一步，确定沿这条路线行进的速度，我们真正需要的是与路径相关的一系列速度，而不是单个速度，该序列称作“速度曲线”；

我们可以使用优化功能为路径选择收到各种限制的良好速度曲线；

通过将路径和速度曲线相结合，可构建车辆行驶轨迹；

路径速度解耦规划又称为EM规划, 你可以在Apollo的Github页面中找到相应的代码实现

## 路径生成与选择

为了在EM规划中生成候选路径，首先将路段分割成单元格，然后对这些单元格的点进行随机采样，通过从每个单元格中取一个点并将点连接；

可以创建多个候选路径，使用成本函数对这些路径进行评估，并选择成本最低的路径；

![](./plan06.png)

## ST图

接下来，选择与路径关联的速度曲线，在ST途中，s表示车辆的纵向位移，t表示时间，ST图上的曲线是对车辆运动的描述，

它描述了车辆在不同时间的位置，可以从曲线的斜率推断速度；

![](./plan07.png)

斜率越陡，表述速度越大；

## 速度规划

为构建最佳速度曲线，我们需要将ST图离散为多个单元格，单元格速度变化，但每个单元格内速度不变，这个方法可简化速度曲线的构建；

并维持曲线的近似度，在ST图中，可以将障碍物绘制为在特定时间段内，阻挡道路的某部分的矩形区域，
为避免碰撞，速度曲线不得与此矩形相交，

![](./plan08.png)

既然有了一张各种单元格被阻挡的ST图，便可以使用优化引擎为该图选择最佳的速度曲线；

![](./plan09.png)

优化算法通过复杂的数学运算来搜索，受到各种限制的低成本解决方案，包括法规，速度限制，距离限制，与障碍物距离，

汽车的物理距离，加速度限制等；

## 优化

EM规划在很大程度上取决于离散化，路径选择涉及将道路划分为单元格，速度曲线构建涉及将ST图划分为单元格；

尽管离散化使这些问题更容易解决，但生产的轨迹并不平滑，为了将离散化解决方案转换为平滑轨迹，可使用“二次规划”技术；

二次规划将平滑的非线性曲线与这些分段式线性段拟合，尽管“二次规划”背后数学原理很复杂，但对我们而言细节不是必需的，

可以只需简单实用集中不同优化包中的一种，比如Apollo退出的运行方案，来生成平滑的轨迹；

一旦路径和速度曲线就绪，便可以用其构建三维轨迹；



## 路径-速度规划的轨迹生成

回顾一下端到端的路径-速度解耦规则，假设在路上行驶，感知系统观察到，一辆缓慢行驶的车辆离我们越来越近，
首先，我们在这辆车周围生成多条候选路线，

使用成本函数进行评估，并选择成本最低的路径，

![](./plan11.png)

然后使用ST图来进行速度规划，根据其他车辆随时间变化的位置阻挡了ST图的部分区域，
优化引擎可帮助我们确定该图的最佳速度曲线，该曲线受制于约束和成本函数，可以使用二次规划让路径和速度曲线变平滑，

![](./plan10.png)

最后，将路径和速度曲线合并构建轨迹，这里轨迹在速度较快时为红色，较慢时为蓝色，
使用该轨迹来安全地绕开其他车辆，并继续我们的行程；

## Lattice规划

通过使用Frenet坐标系，可以将环境投射到纵轴和横轴上，我们的目标是生成三维轨迹：纵向维度、横向维度和时间维度；
可以将三维问题，分解为两个单独的二维问题；

一个二维轨迹是具有时间戳的纵向轨迹，称为ST轨迹；一个是横向偏移，称为SL轨迹；


![](./plan12.png)

Lattice规划具有两个步骤，即分别建立ST和SL轨迹，然后将它们合并；

为生成纵向和横向的二维轨迹，先将初始车辆状态投射到ST坐标系和SL坐标系中；
通过对预选模式中的多个候选最终状态进行采样，来选择最终车辆状态，对于每个候选最终状态，
构建一组轨迹，将我们的车辆从其初始状态转变为最终状态；使用成本函数对这些轨迹进行评估，选择成本最低轨迹

Apollo中的Lattice代码实现：https://github.com/ApolloAuto/apollo/tree/master/modules/planning/planner/lattice

## ST 轨迹的终止状态

分三组：巡航、跟随、停止

巡航：意味着车辆将在完成规划步骤后定速行驶；

跟随：对未知和时间状态进行采样，跟车时需要与前方车保持安全距离；这是速度和加速度取决于跟随的车辆；

停止：只需对汽车何时何地停止进行抽样；

## SL 轨迹的终止状态

采样的是道路上相邻车道中心线周围的位置，为了确保稳定性，汽车驶向的终止状态应该与车道中心一致；

当我们用横向位置与纵向位置作图时，我们想要的候选轨迹应该以车辆与车道对齐并直线行驶而结束；为了达到这种终止状态，车的朝向和位置的一阶和二阶导数，都应该为零，意味着车正沿着车道直行。

![](./plan13.png)

## Polynomial

拟合多项式与评估
我们如何连接初始状态和结束状态?
![](./plan14.png)

## Lattice 规划的轨迹生成

一旦我们同时拥有了ST和SL轨迹，我们就需要将它们重新转换为笛卡尔坐标系，然后将它们相结合；
构建由二维路径点和一维时间戳组成的三维轨迹

![](./plan15.png)

ST轨迹是随时间变化的纵向位移，SL轨迹是纵向轨迹上每个点的横向偏移，由于两个轨迹都有纵坐标S，
所以我们可以通过将其S值进行匹配来合并轨迹；